In [2]:
import numpy as np
import pandas as pd
from joblib import load
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score
import matplotlib.pyplot as plt

In [3]:
DATA_DIR = "/home/welton/data"
PROBS_DIR = f"{DATA_DIR}/normal_probas/split_10"
LABELS_DIR = f"{DATA_DIR}/datasets/labels/split_10"
UPPER_DIR = f"{DATA_DIR}/oracle/upper_bound/normal_probas"
STACKING_DIR = f"{DATA_DIR}/stacking/stacking_output"
CLFS = ["bert", "xlnet", "ktmk", "ktr", "lstmk", "lstr", "ltr"]
N_FOLDS = 10

In [4]:
clfs_stack = '/'.join(sorted([f"{clf}_normal_probas" for clf in CLFS ]))
clfs_stack

'bert_normal_probas/ktmk_normal_probas/ktr_normal_probas/lstmk_normal_probas/lstr_normal_probas/ltr_normal_probas/xlnet_normal_probas'

In [6]:
def load_probs(probs_source: str,
               dataset: str,
               clfs: list,
               fold: int):

    probas = {}
    for clf in clfs:
        probas[clf] = {}
        probas_dir = f"{probs_source}/{dataset}/10_folds/{clf}/{fold}/"
        train_load = np.load(f"{probas_dir}/train.npz")
        test_load = np.load(f"{probas_dir}/test.npz")
        probas[clf]["X_train"] = train_load["X_train"]
        probas[clf]["X_test"] = test_load["X_test"]
    return probas

def load_y(labels_dir: int, dataset: str, fold: int):

    ldir = f"{labels_dir}/{dataset}/{fold}"
    return np.load(f"{ldir}/train.npy"), np.load(f"{ldir}/test.npy")

def load_upper(data_source: str, dataset: str, clfs: str, fold: int):

    upper = {}
    for clf in clfs:
        upper[clf] = {}
        upper_dir = f"{data_source}/{dataset}/10_folds/{clf}/{fold}/"
        upper[clf]["y_train"] = np.load(f"{upper_dir}/train.npz")['y']
        upper[clf]["y_test"] = np.load(f"{upper_dir}/test.npz")['y']
    return upper

def fix_probas(clfs: list,
               y: np.ndarray,
               probas: dict,
               target_class: int):
    
    new_probas = []
    for clf in clfs:
        clf_probas = []
        y_pred = probas[clf]["X_test"].argmax(axis=1)
        for doc_idx, doc_proba in enumerate(probas[clf]["X_test"]):
            
            class_proba = y_pred[doc_idx]
            
            if class_proba != y[doc_idx] and class_proba == target_class:
                clf_probas.append(np.zeros(doc_proba.shape[0]))
            else:
                clf_probas.append(doc_proba)
        
        new_probas.append(np.vstack(clf_probas))
    return np.hstack(new_probas)


In [7]:
meta_layer = "10_folds/logistic_regression/normal_probas"
dataset = "acm"

In [10]:
stats = {}

max_fixes = 0
for fold in np.arange(10):

    y_train, y_test = load_y(LABELS_DIR, "acm", fold)
    probas = load_probs(PROBS_DIR, dataset, CLFS, fold)
    model = load(f"{STACKING_DIR}/{dataset}/{meta_layer}/{clfs_stack}/fold_{fold}/model.joblib")
    
    for label in set(y_test):

        X_test = fix_probas(CLFS, y_test, probas, label)
        y_pred = model.predict(X_test)
        if label not in stats:
            stats[label] = 0
        stats[label] += f1_score(y_test, y_pred, average="macro") / 10


In [11]:
stats

{0: 0.7425590672901641,
 1: 0.7411312885353213,
 2: 0.7390601873416047,
 3: 0.7236793174674726,
 4: 0.745835951842138,
 5: 0.74448181387674,
 6: 0.7406767006666388,
 7: 0.735026491136598,
 8: 0.7397396300233582,
 9: 0.744326554817056,
 10: 0.7372387720808958}

In [ ]:
74,25590672901641
74,11312885353213
73,90601873416047
72,36793174674726
74,5835951842138
74,448181387674
74,06767006666388
73,5026491136598
73,97396300233582
74,4326554817056
73,72387720808958